In [28]:
# Automatic reload of modules when they change
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import numpy as np
import torch
import scipy 
from theseus import SE2

In [30]:
def log_map(x_y_theta):
    # This function is used to convert SE(2) into se(2) by logarithm map
    x, y, theta = x_y_theta[..., 0], x_y_theta[..., 1], x_y_theta[..., 2]
    cosine = torch.cos(theta)
    sine = torch.sin(theta)
    theta = torch.atan2(sine, cosine)
    
    small_theta = theta.abs() < 1e-3
    
    non_zero = torch.ones(1, dtype=theta.dtype, device=theta.device)
    sine_nz = torch.where(small_theta, non_zero, sine)
    
    half_theta_by_tan_half_theta = (
        0.5
        * (1 + cosine)
        * torch.where(small_theta, 1 + sine**2 / 6, theta / sine_nz)
    )
    half_theta = 0.5 * theta
    
    # Compute the translation
    ux = half_theta_by_tan_half_theta * x + half_theta * y
    uy = half_theta_by_tan_half_theta * y - half_theta * x
    
    return torch.stack((ux, uy, theta), dim=-1)

In [31]:
def T_matrix(x_y_theta):
    matrix_shape = x_y_theta.shape[:-1] + (3, 3)
    # Initialize the identity matrix
    T = np.zeros(matrix_shape, dtype=x_y_theta.dtype)
    T[..., 0, 0] = 1
    T[..., 1, 1] = 1
    T[..., 2, 2] = 1
    
    # Compute the rotation matrix
    theta = x_y_theta[..., 2]
    cosine = np.cos(theta)
    sine = np.sin(theta)
    T[..., 0, 0] = cosine
    T[..., 0, 1] = -sine
    T[..., 1, 0] = sine
    T[..., 1, 1] = cosine
    
    # Compute the translation matrix
    x, y = x_y_theta[..., 0], x_y_theta[..., 1]
    T[..., 0, 2] = x
    T[..., 1, 2] = y
    
    return T

In [36]:
vec = np.random.uniform(low = -np.pi, high = np.pi, size=(10, 3))
vec_torch = torch.from_numpy(vec)

se2 = SE2(x_y_theta=vec_torch)

T_list = T_matrix(vec)

t_scipy = np.zeros_like(vec)
for i, T in enumerate(T_list):
    t_hat = scipy.linalg.logm(T)
    t_scipy[i] = np.array([t_hat[0, 2], t_hat[1, 2], t_hat[1, 0]])

t_thesus = se2.log_map().numpy()
t_imp = log_map(vec_torch).numpy()

In [37]:
print(t_scipy)

[[-0.9902846  -2.3865215   1.0502493 ]
 [ 1.32199835 -2.6564474   1.36559603]
 [-2.87696039  0.60465853 -0.22798945]
 [ 2.74885381 -0.97105584 -0.62022277]
 [ 0.91989462 -1.05373842 -0.70613813]
 [-0.56962287  3.4600836   1.41243681]
 [-3.47329162  0.08442632  2.79023557]
 [-1.29801264 -1.419597    0.89781615]
 [ 1.15252262 -3.00301786  0.17998338]
 [-0.20106348  1.30075144 -1.94377497]]


In [38]:
error1 = np.abs(t_scipy - t_imp)
error2 = np.abs(t_scipy - t_thesus)
error3 = np.abs(t_imp - t_thesus)

In [39]:
print(error1, error2, error3)

[[2.22044605e-16 0.00000000e+00 0.00000000e+00]
 [6.66133815e-16 8.88178420e-16 0.00000000e+00]
 [0.00000000e+00 1.11022302e-16 5.55111512e-17]
 [4.44089210e-16 1.11022302e-16 3.33066907e-16]
 [2.22044605e-16 2.22044605e-16 1.11022302e-16]
 [1.11022302e-16 4.44089210e-16 2.22044605e-16]
 [0.00000000e+00 5.55111512e-16 8.88178420e-16]
 [4.44089210e-16 0.00000000e+00 3.33066907e-16]
 [4.44089210e-16 4.44089210e-16 8.32667268e-17]
 [2.22044605e-16 6.66133815e-16 2.22044605e-16]] [[2.22044605e-16 0.00000000e+00 0.00000000e+00]
 [6.66133815e-16 8.88178420e-16 0.00000000e+00]
 [0.00000000e+00 1.11022302e-16 5.55111512e-17]
 [4.44089210e-16 1.11022302e-16 3.33066907e-16]
 [2.22044605e-16 2.22044605e-16 1.11022302e-16]
 [1.11022302e-16 4.44089210e-16 2.22044605e-16]
 [0.00000000e+00 5.55111512e-16 8.88178420e-16]
 [4.44089210e-16 0.00000000e+00 3.33066907e-16]
 [4.44089210e-16 4.44089210e-16 8.32667268e-17]
 [2.22044605e-16 6.66133815e-16 2.22044605e-16]] [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [

In [2]:
import torch

a = torch.randn(64, 32, 3, 1)
b = torch.randn(64, 32, 3, 3)


r = torch.einsum('bmij, bmjk->bmik',
                torch.einsum('bmij, bmik->bmjk', a, b),
                a).unsqueeze(-1).unsqueeze(-1)

In [7]:
i = 8
j = 31
a[i,j].T @ b[i,j] @ a[i,j] - r[i,j]

tensor([[[[0.]]]])